In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
ts = TimeSeries(key='2J2BP2H90CUIBWSJ',output_format='pandas')

In [2]:
DIS_data, metadata=ts.get_daily('DIS',outputsize='full')
DIS_data.describe()

,1. open,2. high,3. low,4. close,5. volume
count,5226.000000,5226.000000,5226.000000,5226.000000,5.226000e+03
mean,56.091725,56.611333,55.535068,56.095625,9.433350e+06
std,37.213090,37.432899,36.931040,37.185586,5.785021e+06
min,13.800000,14.100000,13.480000,13.770000,1.487900e+06
25%,27.270000,27.650000,26.965000,27.292500,6.120075e+06
50%,35.500000,35.945000,35.110000,35.490000,8.003400e+06
75%,95.302500,96.265000,94.445000,95.187500,1.098850e+07
max,152.940000,153.410000,151.150000,151.640000,1.166250e+08


In [3]:
DIS_data.to_csv('DIS_data.csv')

In [4]:
DIS_data['date'] = DIS_data.index

In [5]:
DIS_data

,1. open,2. high,3. low,4. close,5. volume,date
date,,,,,,
2020-08-07,130.22,131.94,128.66,129.93,16088579.0,2020-08-07
2020-08-06,126.96,131.63,125.61,130.82,27237996.0,2020-08-06
2020-08-05,123.37,130.31,123.28,127.61,53122394.0,2020-08-05
2020-08-04,115.96,118.35,115.76,117.29,17293923.0,2020-08-04
2020-08-03,116.17,117.12,115.35,116.35,10229172.0,2020-08-03
...,...,...,...,...,...,...
1999-11-05,25.50,25.50,24.06,24.31,26165000.0,1999-11-05
1999-11-04,27.75,29.00,26.00,26.50,13492400.0,1999-11-04
1999-11-03,27.38,27.38,26.69,26.88,8196800.0,1999-11-03


In [6]:
DIS_training_complete = pd.read_csv('/Users/kelbywilson/Desktop/project_3/DIS_data.csv')

In [10]:
DIS_training_processed = DIS_training_complete.iloc[:, 1:2].values

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

DIS_training_scaled = scaler.fit_transform(DIS_training_processed)

In [14]:
features_set = []
labels = []
for i in range(60, 5226):
    features_set.append(DIS_training_scaled[i-60:i, 0])
    labels.append(DIS_training_scaled[i, 0])

In [15]:
# Convert both the feature_set and the labels list to the numpy array before we can use it for training
features_set, labels = np.array(features_set), np.array(labels)

In [16]:
# In order to train LSTM on our data, we need to convert our data into the shape accepted by the LSTM.
# We need to convert our data into three-dimensional format
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [18]:
model = Sequential()

In [19]:
model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [20]:
# Add dropout layer to avoid overfitting of the data
model.add(Dropout(0.2))

In [21]:
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

In [22]:
# To make our model more robust we add a dense layer
model.add(Dense(units = 1))

In [23]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [24]:
model.fit(features_set, labels, epochs = 50, batch_size = 32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 5166 samples
Epoch 1/50
5166/5166 [==============================] - 26s 5ms/sample - loss: 0.0085
Epoch 2/50
5166/5166 [==============================] - 25s 5ms/sample - loss: 0.0021
Epoch 3/50
5166/5166 [==============================] - 27s 5ms/sample - loss: 0.0020
Epoch 4/50
5166/5166 [==============================] - 25s 5ms/sample - loss: 0.0017
Epoch 5/50
5166/5166 [==============================] - 24s 5ms/sample - loss: 0.0017
Epoch 6/50
5166/5166 [==============================] - 24s 5ms/sample - loss: 0.0017
Epoch 7/50
5166/5166 [==============================] - 23s 5ms/sample - loss: 0.0015
Epoch 8/50
5166/5166 [==============================] - 23s 5ms/sample - loss: 0.0013
Epoch 9/50
5166/5166 [==============================] - 23s 5ms/sample - loss: 0.0013
Epoch 10/50
5166/5166 [==============================] - 24s 5ms/sample - loss: 0.0011
Epoch 11/50
5166/5166 [